# Diluted Unitary Fitting with KL-Div

In [1]:
import sys
sys.path.insert(0, '../../../src_tf/')

import numpy as np
import tensorflow as tf
import random
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

from quantum_tools import generate_unitary
from quantum_channel import channel_spectrum
from loss_functions import ProbabilityMSE, channel_fidelity_loss, SpectrumDistance
from optimization import ModelQuantumMap, Logger
from kraus_channels import KrausMap, DilutedKrausMap
from lindblad_channels import CompactLindbladMap
from spam import InitialState, POVM, SPAM, CorruptionMatrix
from utils import saver
from synthetic_data import generate_spam_data, generate_spam_benchmark, generate_map_data
from spectrum import mean_spacing
from utils import loader

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
def optimize_annulus_distance(spectrum_target = None, 
                              d = None, 
                              c_start = 0.5,
                              rank_start = 1,
                              rank_step = 1,
                              num_iter = 1000,
                              lr = 0.001,
                              verbose = True): 
    
    model_list = []
    distance_list = []
    counter = 0

    sigma = mean_spacing(spectrum_target)
    
    for rank in range(rank_start, d**2+rank_step, rank_step): 
        model = ModelQuantumMap(channel = DilutedKrausMap(U=generate_unitary(d), 
                                                          c=c_start, 
                                                          kraus_part=KrausMap(d, 
                                                                              rank=rank, 
                                                                              trainable=False),
                                                     ),
                            loss_function = SpectrumDistance(sigma=sigma, k = None, remove_shift=True),
                            optimizer = tf.optimizers.Adam(learning_rate=lr),
                            logger = Logger(loss_function = SpectrumDistance(sigma=sigma, k = None), 
                                            verbose=verbose,
                                            N=0),
                            )
        model.train(inputs = None,
                    targets = [spectrum_target],
                    num_iter = num_iter,
                    verbose = verbose,
                   )

        c_start = model.channel.c.numpy()
        model_list.append(model)
        distance_list.append(model.logger.loss_train_list[-1])
        
        idx = np.argmin(distance_list)
        if distance_list[-1] > distance_list[idx]:
            counter += 1
        else:
            counter = 0
            
        if counter > 1:
            print("Done!")
            break

    distance_best = distance_list[idx]
    model_best = model_list[idx]

    return distance_best, model_best

def fit_diluted_unitary(spectrum_target_list, 
                        d, 
                        rank_start, 
                        rank_step, 
                        num_iter = 1000,
                        lr = 0.001):
    
    distance_best_list = []
    model_best_list = []
    for spectrum_target in tqdm(spectrum_target_list):
        distance_best, model_best = optimize_annulus_distance(spectrum_target,
                                                              d,
                                                              rank_start,
                                                              rank_step,
                                                              num_iter = num_iter,
                                                              lr = lr)
        distance_best_list.append(distance_best)
        model_best_list.append(model_best)
    
    for model in model_best_list:
        model.optimizer = None
        
    return distance_best_list, model_best_list

## Four Qubits, Repetition

In [3]:
n = 4
d = 2**n

path1 = "../../../data/"
path2 = "data/"

[model1, model2, model3, model4, model5] = loader(path1 + "belem_4qubit_repeated.model")

spectrum1 = channel_spectrum(model1.channel, keep_unity=False)
spectrum2 = channel_spectrum(model2.channel, keep_unity=False) 
spectrum3 = channel_spectrum(model3.channel, keep_unity=False)
spectrum4 = channel_spectrum(model4.channel, keep_unity=False)

In [5]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_8layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

0.020706850118542272 None
-0.015935838617283864 None
-0.015935960828195907 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.016271402929545875 None
-0.018831352074741632 None
-0.01883140882204054 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.023155704393221045 None
-0.023180141394803636 None
-0.023180143418887002 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.021891425091240695 None
-0.021989046962398708 None
-0.021989049226293328 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.019215331997178786 None
-0.01924885362598449 None
-0.019248855122972912 None
Done!


In [15]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = 0.5,
                                                      rank_start = 9, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_16layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

-0.08621400256079964 None
-0.4973633470635136 None
-0.5004010019311933 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.501572482120854 None
-0.5015857457281238 None
-0.5015857461959112 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.5031754013005294 None
-0.5032273035081171 None
-0.5032273046653729 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.503152822606019 None
-0.503183076601281 None
-0.503183076890922 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.5030958346895338 None
-0.5031350878790081 None
-0.5031350880229138 None
Done!


In [5]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = 0.18,
                                                      rank_start = 45, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_24layer.model")

Done!


In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = 0.12, 
                                                      rank_start = 65, 
                                                      rank_step = 1,
                                                      num_iter = 200,
                                                      lr=0.01,
                                                      verbose=True)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_KL_4qubit_32layer.model")

  0%|          | 0/200 [00:00<?, ?it/s]

-0.01714612348200053 None
-0.029903951525293335 None
-0.029904290731283506 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.030258275837993745 None
-0.03068366825464161 None
-0.030683676586882785 None


  0%|          | 0/200 [00:00<?, ?it/s]

...

-0.03512187649144033 None
-0.03587942386085753 None
-0.035879439686292244 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.029742881445571898 None
-0.029894725503232203 None
-0.029894728277279348 None


  0%|          | 0/200 [00:00<?, ?it/s]

-0.033977731422910426 None
-0.03397999195986155 None
-0.03397999196086436 None
Done!


## Four Qubits, Delay

In [18]:
n = 4
d = 2**n

path1 = "../../../data/"
path2 = "data/"

[model1, model2, model3, model4, model5, model6, model7, model8, model9] = loader(path1 + "belem_4qubit_delay.model")

spectrum1 = channel_spectrum(model1.channel, keep_unity=False)
spectrum2 = channel_spectrum(model2.channel, keep_unity=False) 
spectrum3 = channel_spectrum(model4.channel, keep_unity=False)
spectrum4 = channel_spectrum(model6.channel, keep_unity=False)

In [13]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum1, 
                                                      d = d,
                                                      c_start = 0.4,
                                                      rank_start = 6, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_0delay.model")

0.014205054440732518 None
0.008845493580107968 None
0.0041204841226500505 None
0.005524199912069497 None
0.005178203361651448 None
0.005439743360235921 None
0.005972243073423356 None
0.008965927547632122 None
Done!


In [15]:
distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum2, 
                                                      d = d,
                                                      c_start = 0.3,
                                                      rank_start = 10, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_5delay.model")

0.01848348460790743 None
0.014186931516701865 None
0.009715298239793096 None
0.007303294242447408 None
0.005723892968955122 None
0.004077427925329443 None
0.0025232176524793357 None
0.001863246488379957 None
0.003712720919869558 None
0.003806595261150243 None
0.0063913662393518395 None
0.006416087967554447 None
0.009325553550093457 None
Done!


In [16]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum3, 
                                                      d = d, 
                                                      c_start = 0.18,
                                                      rank_start = 45, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_10delay.model")

0.009643465250119252 None
0.010842895343623385 None
0.012202345632732953 None
0.011250103925621733 None
0.011827404001205143 None
0.01217502180153485 None
Done!


In [19]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

distance_best, model_best = optimize_annulus_distance(spectrum_target = spectrum4, 
                                                      d = d,
                                                      c_start = 0.12, 
                                                      rank_start = 65, 
                                                      rank_step = 1,
                                                      num_iter = 100,
                                                      lr=0.05,
                                                      verbose=False)

model_best.optimizer = None
saver([distance_best, model_best], path2 + "diluted_fitting_4qubit_12_5delay.model")

0.012446957367763904 None
0.012270846412483716 None
0.01264303974927565 None
0.012950804567063212 None
0.012072362215487242 None
0.013101705753887218 None
0.013578943158130453 None
0.013536197943886753 None
Done!
